In [1]:
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

os.makedirs("images", exist_ok=True)

In [10]:
N_EPOCHS = 100    # number of epochs of training
BATCH_SIZE = 200   # size of the batches
LR = 2e-5         # adam: learning rate
B1 = 0.5          # adam: decay of first order momentum of gradient
B2 = 0.999        # adam: decay of first order momentum of gradient

N_CPU = 11         # number of cpu threads to use during batch generation
LATENT_DIM = 100  # dimensionality of the latent space
IMG_SIZE = 28     # size of each image dimension
CHANNELS = 1      # number of image channels
SAMPLE_INTERVAL = 400 # interval betwen image samples

In [3]:
img_shape = (CHANNELS, IMG_SIZE, IMG_SIZE)
cuda = True if torch.cuda.is_available() else False
print("CUDA:", cuda)

CUDA: True


In [4]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(LATENT_DIM, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape)
        return img

In [5]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)

        return validity

In [6]:
# Loss function
adversarial_loss = torch.nn.BCELoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

In [7]:
# Configure data loader
# os.makedirs("../PyTorch/MNIST", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../PyTorch/sentdex/",
        train=True,
        download=True,
        transform=transforms.Compose(
            [
                transforms.Resize(IMG_SIZE), 
                transforms.ToTensor(), 
                transforms.Normalize([0.5], [0.5])
            ]
        ),
    ),
    batch_size=BATCH_SIZE,
    shuffle=True,
)

In [8]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=LR, betas=(B1, B2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=LR, betas=(B1, B2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [11]:
for epoch in range(N_EPOCHS):
    for i, (imgs, _) in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], LATENT_DIM))))

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        batches_done = epoch * len(dataloader) + i
        if batches_done % SAMPLE_INTERVAL == 0:
            save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)
    
    print(
            "[Epoch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, N_EPOCHS, d_loss.item(), g_loss.item())
        )

[Epoch 0/100] [D loss: 0.244370] [G loss: 1.529977]
[Epoch 1/100] [D loss: 0.246279] [G loss: 1.636408]
[Epoch 2/100] [D loss: 0.250268] [G loss: 1.398606]
[Epoch 3/100] [D loss: 0.283204] [G loss: 1.435591]
[Epoch 4/100] [D loss: 0.254047] [G loss: 1.451044]
[Epoch 5/100] [D loss: 0.262585] [G loss: 1.521933]
[Epoch 6/100] [D loss: 0.241266] [G loss: 1.478504]
[Epoch 7/100] [D loss: 0.350346] [G loss: 2.155133]
[Epoch 8/100] [D loss: 0.315411] [G loss: 1.142281]
[Epoch 9/100] [D loss: 0.248699] [G loss: 1.635190]
[Epoch 10/100] [D loss: 0.351436] [G loss: 1.754505]
[Epoch 11/100] [D loss: 0.313858] [G loss: 1.461320]
[Epoch 12/100] [D loss: 0.410480] [G loss: 0.770485]
[Epoch 13/100] [D loss: 0.288312] [G loss: 1.743319]
[Epoch 14/100] [D loss: 0.335282] [G loss: 1.078215]
[Epoch 15/100] [D loss: 0.266451] [G loss: 1.503704]
[Epoch 16/100] [D loss: 0.343639] [G loss: 1.456352]
[Epoch 17/100] [D loss: 0.299475] [G loss: 2.226129]
[Epoch 18/100] [D loss: 0.294989] [G loss: 1.805994]
[Ep